# run THDM-II model

The idea is run the model

In [1]:
import numpy as np
import pandas as pd
import subprocess
import time

%matplotlib inline
import matplotlib.pyplot as plt 

Read Fredy data:

In [2]:
xd0 = pd.read_csv('data.txt')

In [10]:
xd0[:2]

,L1,L2,L3,L4,L5,M12
0,0.163834,0.307072,0.126198,-0.35795,-0.207587,-173780.0
1,0.163834,0.307072,0.126198,-0.35795,-0.207587,-177828.0


In [7]:
import pyslha
import pyTHDM_LesHouches_generator
import sys

Cambiar valores dentro del archivo LesHouches

In [13]:
xd0[:3]

,L1,L2,L3,L4,L5,M12
0,0.163834,0.307072,0.126198,-0.35795,-0.207587,-173780.0
1,0.163834,0.307072,0.126198,-0.35795,-0.207587,-177828.0
2,0.163834,0.307072,0.126198,-0.35795,-0.207587,-181970.0


In [14]:
for i in range(0,3):
    print(xd0['L1'][i])

0.163834
0.163834
0.163834


cantidad de datos

In [23]:
xd0.shape

(630, 6)

In [24]:
xd0.shape[0]

630

In [26]:
x=[]

Num = 5
#Num = xd0.shape[0]

for i in range (0,Num):
       
    L1 = xd0['L1'][i] 
    L2 = xd0['L2'][i]
    L3 = xd0['L3'][i]
    L4 = xd0['L4'][i]
    L5 = xd0['L5'][i]
    M12 = xd0['M12'][i]
    tb = 2.00569

    #Open the file that will created the LesHouches file
    xdict = pyTHDM_LesHouches_generator.buildSLHAinFile()

    xdict.blocks['MINPAR'].entries[1]='%.6E    # Lambda1Input'%L1
    xdict.blocks['MINPAR'].entries[2]='%.6E    # Lambda2Input'%L2
    xdict.blocks['MINPAR'].entries[3]='%.6E    # Lambda3Input'%L3
    xdict.blocks['MINPAR'].entries[4]='%.6E    # Lambda4Input'%L4
    xdict.blocks['MINPAR'].entries[5]='%.6E    # Lambda5Input'%L5
    xdict.blocks['MINPAR'].entries[9]='%.6E    # M12inputt'%M12
    xdict.blocks['MINPAR'].entries[10]='%.6E    # TanBeta' %tb

    #Write and created the Leshouches file generator
    pyslha.writeSLHAFile('LesHouches.in.THDMII_low',xdict)  

    #run SPheno
    spheno = subprocess.getoutput('~/tmp/SPheno-4.0.4/bin/./SPhenoTHDMII LesHouches.in.THDMII_low')    
    SPheno_output = subprocess.getoutput('cat SPheno.spc.THDMII')

    #capturo algo de spheno

    #MASAS
    mh1 = eval(SPheno_output.split('#   PDG code')[1].split()[3])
    mh2 = eval(SPheno_output.split('#   PDG code')[1].split()[7])
    mAh = eval(SPheno_output.split('#   PDG code')[1].split()[11])
    mHm = eval(SPheno_output.split('#   PDG code')[1].split()[15])
    #print("========== MASAS ==============")
    #print("mh1=",mh1,"mh2=",mh2,"mAh=",mAh,"mHm=",mHm)
    #print("===============================\n")

    #Couplings
    h1bb = eval(SPheno_output.split('HiggsBoundsInputHiggsCouplingsFermions')[1].split()[1])
    h1tt = eval(SPheno_output.split('# h_1 s s coupling')[1].split()[0])
    h1tautau = eval(SPheno_output.split('# h_1 c c coupling')[1].split()[0])
    #print("============ Couplings ================")
    #print("chbb=",h1bb,"chtt=",h1tt,"chtautau=",h1tautau)
    #print("============================\n")

    #RUN Higgsbounds

    #copio los archivos a la carpeta Input_files
    subprocess.getoutput('cp Messages.out effC.dat BR* Key.dat LEP* LesHouches.in.THDMII_low MH* SPheno.spc.THDMII Input_Files/')
    #run HiggsBounds
    hb = subprocess.getoutput('~/Work/HiggsBounds-4.3.1/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')

    #Capturo lo que necesito del archivo de resultados
    HB_results = subprocess.getoutput('cat Input_Files/HiggsBounds_results.dat')
    HBresult = eval(HB_results.split('cols:')[1].split()[15])
    #print("HBresult=",HBresult)
    
    x.append([L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,HBresult])

x=np.asarray(x)
xd=pd.DataFrame(x,columns=['L1','L2','L3','L4','L5','M12','tb','mh1','mh2','mAh','mHm','h1bb','h1tt','h1tautau','HBresult'])   

In [28]:
xd

,L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,HBresult
0,0.163834,0.307072,0.126198,-0.35795,-0.207587,-173780.0,2.00569,123.925494,673.204942,669.048957,672.405967,1.075055,0.980451,1.075055,1.0
1,0.163834,0.307072,0.126198,-0.35795,-0.207587,-177828.0,2.00569,123.982531,680.682044,676.582440,679.902254,1.073374,0.980909,1.073374,1.0
2,0.163834,0.307072,0.126198,-0.35795,-0.207587,-181970.0,2.00569,124.038260,688.249167,684.205007,687.488013,1.071731,0.981356,1.071731,1.0
3,0.163834,0.307072,0.126198,-0.35795,-0.207587,-186209.0,2.00569,124.092720,695.908766,691.919141,695.165718,1.070124,0.981791,1.070124,1.0
4,0.163834,0.307072,0.126198,-0.35795,-0.207587,-190546.0,2.00569,124.145924,703.659597,699.723605,702.934137,1.068552,0.982217,1.068552,1.0


In [29]:
xd.to_csv('salida.csv')

In [31]:
#old
#Open the file that will created the LesHouches file
xdict = pyTHDM_LesHouches_generator.buildSLHAinFile()

#Created a benchmark point                                                                            
L1 = 0.121124
L2 = 0.288218
L3 = 0.049769
L4 = -0.0415422
L5 = -0.381272
M12 = -85113.8
tb = 2.00569

xdict.blocks['MINPAR'].entries[1]='%.6E    # Lambda1Input'%L1
xdict.blocks['MINPAR'].entries[2]='%.6E    # Lambda2Input'%L2
xdict.blocks['MINPAR'].entries[3]='%.6E    # Lambda3Input'%L3
xdict.blocks['MINPAR'].entries[4]='%.6E    # Lambda4Input'%L4
xdict.blocks['MINPAR'].entries[5]='%.6E    # Lambda5Input'%L5
xdict.blocks['MINPAR'].entries[9]='%.6E    # M12inputt'%M12
xdict.blocks['MINPAR'].entries[10]='%.6E    # TanBeta' %tb

#Write and created the Leshouches file generator
pyslha.writeSLHAFile('LesHouches.in.THDMII_low',xdict)  

#run SPheno
spheno = subprocess.getoutput('~/tmp/SPheno-4.0.4/bin/./SPhenoTHDMII LesHouches.in.THDMII_low')    
SPheno_output = subprocess.getoutput('cat SPheno.spc.THDMII')

#capturo algo de spheno

#MASAS
mh1 = eval(SPheno_output.split('#   PDG code')[1].split()[3])
mh2 = eval(SPheno_output.split('#   PDG code')[1].split()[7])
mAh = eval(SPheno_output.split('#   PDG code')[1].split()[11])
mHm = eval(SPheno_output.split('#   PDG code')[1].split()[15])
print("========== MASAS ==============")
print("mh1=",mh1,"mh2=",mh2,"mAh=",mAh,"mHm=",mHm)
print("===============================\n")

#Couplings
h1bb = eval(SPheno_output.split('HiggsBoundsInputHiggsCouplingsFermions')[1].split()[1])
h1tt = eval(SPheno_output.split('# h_1 s s coupling')[1].split()[0])
h1tautau = eval(SPheno_output.split('# h_1 c c coupling')[1].split()[0])
print("============ Couplings ================")
print("chbb=",h1bb,"chtt=",h1tt,"chtautau=",h1tautau)
print("============================\n")

#RUN Higgsbounds

#copio los archivos a la carpeta Input_files
subprocess.getoutput('cp Messages.out effC.dat BR* Key.dat LEP* LesHouches.in.THDMII_low MH* SPheno.spc.THDMII Input_Files/')
#run HiggsBounds
hb = subprocess.getoutput('~/Work/HiggsBounds-5.3.2beta/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')

#Capturo lo que necesito del archivo de resultados
HB_results = subprocess.getoutput('cat Input_Files/HiggsBounds_results.dat')
HBresult = eval(HB_results.split('cols:')[1].split()[15])
print("HBresult=",HBresult)


========== MASAS ==============
mh1= 120.444966 mh2= 478.73116 mAh= 485.783658 mHm= 475.195829

============ Couplings ================
chbb= 1.14038173 chtt= 0.9619293 chtautau= 1.14038173

HBresult= 0


In [32]:
hb = subprocess.getoutput('~/Work/HiggsBounds-5.3.2beta/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')


In [34]:
hb.splitlines()

['',
 ' ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~',
 ' ~                                                        ~',
 ' ~                   HiggsBounds 5.3.2beta                ~',
 ' ~                                                        ~',
 ' ~     Philip Bechtle, Daniel Dercks, Sven Heinemeyer,    ~',
 ' ~      Tobias Klingl, Tim Stefaniak, Georg Weiglein      ~',
 ' ~                                                        ~',
 ' ~            arXiv:0811.4169, arXiv:1102.1898,           ~',
 ' ~            arXiv:1301.2345, arXiv:1311.0055            ~',
 ' ~            arXiv:1507.06706,                           ~',
 ' ~            http://higgsbounds.hepforge.org             ~',
 ' ~                                                        ~',
 ' ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~',
 '',
 ' HiggsBounds collects together results from ',
 '',
 '     * the LEP collaborations and LEP Higgs Working Group',
 '     * the CDF and D0 Collaborations',
 